# Spark DataFrames Project Exercise

Let's get some quick practice with your new Spark DataFrame skills, you will be asked some basic questions about some stock market data, in this case Walmart Stock from the years 2012-2017. This exercise will just ask a bunch of questions, unlike the future machine learning exercises, which will be a little looser and be in the form of "Consulting Projects", but more on that later!

For now, just answer the questions and complete the tasks below.

#### Use the walmart_stock.csv file to Answer and complete the  tasks below!

#### Start a simple Spark Session

In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('exercise').getOrCreate()

#### Load the Walmart Stock CSV File, have Spark infer the data types.

In [7]:
df = spark.read.csv(inferSchema=True, header = True, path = '/FileStore/tables/walmart_stock.csv')

#### What are the column names?

In [9]:
df.columns

Out[3]: ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

#### What does the Schema look like?

In [11]:
df.printSchema()

root
-- Date: string (nullable = true)
-- Open: double (nullable = true)
-- High: double (nullable = true)
-- Low: double (nullable = true)
-- Close: double (nullable = true)
-- Volume: integer (nullable = true)
-- Adj Close: double (nullable = true)

#### Print out the first 5 columns.

In [13]:
df.show(5)

+----------+------------------+---------+---------+------------------+--------+------------------+
 Date| Open| High| Low| Close| Volume| Adj Close|
+----------+------------------+---------+---------+------------------+--------+------------------+
2012-01-03| 59.970001|61.060001|59.869999| 60.330002|12668800|52.619234999999996|
2012-01-04|60.209998999999996|60.349998|59.470001|59.709998999999996| 9593300| 52.078475|
2012-01-05| 59.349998|59.619999|58.369999| 59.419998|12768200| 51.825539|
2012-01-06| 59.419998|59.450001|58.869999| 59.0| 8069400| 51.45922|
2012-01-09| 59.029999|59.549999|58.919998| 59.18| 6679300|51.616215000000004|
+----------+------------------+---------+---------+------------------+--------+------------------+
only showing top 5 rows

#### Use describe() to learn about the DataFrame.

In [15]:
df.describe().show()

+-------+----------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
summary| Date| Open| High| Low| Close| Volume| Adj Close|
+-------+----------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
 count| 1258| 1258| 1258| 1258| 1258| 1258| 1258|
 mean| null| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
 stddev| null| 6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991| 4519780.8431556|6.722609449996857|
 min|2012-01-03|56.389998999999996| 57.060001| 56.299999| 56.419998| 2094900| 50.363689|
 max|2016-12-30| 90.800003| 90.970001| 89.25| 90.470001| 80898100|84.91421600000001|
+-------+----------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+

## Bonus Question!
#### There are too many decimal places for mean and stddev in the describe() dataframe. Format the numbers to just show up to two decimal places. Pay careful attention to the datatypes that .describe() returns, we didn't cover how to do this exact formatting, but we covered something very similar. [Check this link for a hint](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.cast)

If you get stuck on this, don't worry, just view the solutions.

In [17]:
from pyspark.sql.functions import format_number

df02 = df.describe()

df02.printSchema()

root
-- summary: string (nullable = true)
-- Date: string (nullable = true)
-- Open: string (nullable = true)
-- High: string (nullable = true)
-- Low: string (nullable = true)
-- Close: string (nullable = true)
-- Volume: string (nullable = true)
-- Adj Close: string (nullable = true)

In [18]:
from pyspark.sql.types import StructField,StringType,IntegerType,StructType,DoubleType

In [19]:
df3 = df02.select(df02['summary'], 
             df02['Date'],
             format_number(df02.Open.cast(DoubleType()),2).alias('Open'),
             format_number(df02.High.cast(DoubleType()),2).alias('High'),
             format_number(df02.Low.cast('double'),2).alias('Low'),
             format_number(df02.Close.cast('double'),2).alias('Close'),
             df02.Volume.cast('integer'),
             format_number(df02['Adj Close'].cast('double'),2).alias('Close'))

In [20]:
df3.show()

+-------+----------+--------+--------+--------+--------+--------+--------+
summary| Date| Open| High| Low| Close| Volume| Close|
+-------+----------+--------+--------+--------+--------+--------+--------+
 count| 1258|1,258.00|1,258.00|1,258.00|1,258.00| 1258|1,258.00|
 mean| null| 72.36| 72.84| 71.92| 72.39| 8222093| 67.24|
 stddev| null| 6.77| 6.77| 6.74| 6.76| 4519780| 6.72|
 min|2012-01-03| 56.39| 57.06| 56.30| 56.42| 2094900| 50.36|
 max|2016-12-30| 90.80| 90.97| 89.25| 90.47|80898100| 84.91|
+-------+----------+--------+--------+--------+--------+--------+--------+

#### Create a new dataframe with a column called HV Ratio that is the ratio of the High Price versus volume of stock traded for a day.

In [22]:
df04 = df.withColumn('HV Ratio', df.High / df.Volume)

df04.select(['Date','HV Ratio']).show(5)

+----------+--------------------+
 Date| HV Ratio|
+----------+--------------------+
2012-01-03|4.819714653321546E-6|
2012-01-04|6.290848613094555E-6|
2012-01-05|4.669412994783916E-6|
2012-01-06|7.367338463826307E-6|
2012-01-09|8.915604778943901E-6|
+----------+--------------------+
only showing top 5 rows

#### What day had the Peak High in Price?

In [24]:
df.groupBy('Date').max('High').orderBy('max(High)', ascending = False).head()[0]

Out[12]: '2015-01-13'

#### What is the mean of the Close column?

In [26]:
df.groupBy().mean('Close').show()

+-----------------+
 avg(Close)|
+-----------------+
72.38844998012726|
+-----------------+

#### What is the max and min of the Volume column?

In [28]:
# Describe method

df.describe().filter(" summary = 'max' or summary = 'min'").select('summary','Volume').show()

+-------+--------+
summary| Volume|
+-------+--------+
 min| 2094900|
 max|80898100|
+-------+--------+

In [29]:
from pyspark.sql.functions import max,min

df.select(max("Volume"),min("Volume")).show()

+-----------+-----------+
max(Volume)|min(Volume)|
+-----------+-----------+
 80898100| 2094900|
+-----------+-----------+

In [30]:
# Aggregation

df.agg(min('Volume'), max('Volume')).show()

+-----------+-----------+
min(Volume)|max(Volume)|
+-----------+-----------+
 2094900| 80898100|
+-----------+-----------+

In [31]:
# MIN

df.groupBy().min('Volume').show()

+-----------+
min(Volume)|
+-----------+
 2094900|
+-----------+

In [32]:
# MAX with GroupBy object

df.groupBy().max('Volume').show()

+-----------+
max(Volume)|
+-----------+
 80898100|
+-----------+

In [33]:
# MAX with filtering on describe()

df05 = df.describe()

df05.filter("summary = 'max'").select('Volume').show()

+--------+
 Volume|
+--------+
80898100|
+--------+

In [34]:
# .where is an alias for .filter

df05 = df.describe()

df05.where("summary = 'max'").select('Volume').show()

+--------+
 Volume|
+--------+
80898100|
+--------+

In [35]:
# MAX with aggregation object

df.agg({'Volume':'max'}).show()

+-----------+
max(Volume)|
+-----------+
 80898100|
+-----------+

#### How many days was the Close lower than 60 dollars?

In [37]:
df.filter(df.Close < 60).count()

Out[28]: 81

#### What percentage of the time was the High greater than 80 dollars ?
#### In other words, (Number of Days High>80)/(Total Days in the dataset)

In [39]:
from pyspark.sql.functions import countDistinct

result01 = df.filter(df.High > 80).count()

result02 = df.agg(countDistinct('Date')).collect()[0][0]

(result01/result02)*100

Out[62]: 9.141494435612083

In [40]:
# 9.14 percent of the time it was over 80
# Many ways to do this
(df.filter(df["High"]>80).count()*1.0/df.count())*100

Out[126]: 9.141494435612083

#### What is the Pearson correlation between High and Volume?
#### [Hint](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameStatFunctions.corr)

In [42]:
df.corr('High','Volume', method = 'pearson')

Out[63]: -0.3384326061737161

In [43]:
from pyspark.sql.functions import corr
df.select(corr("High","Volume")).show()

+-------------------+
 corr(High, Volume)|
+-------------------+
-0.3384326061737161|
+-------------------+

#### What is the max High per year?

In [45]:
# orderBy(col, ascending = False|0 ou True|1)

from pyspark.sql.functions import year

df.groupBy(year('Date')).max('High').orderBy('year(Date)', ascending = 0).show()

+----------+---------+
year(Date)|max(High)|
+----------+---------+
 2016|75.190002|
 2015|90.970001|
 2014|88.089996|
 2013|81.370003|
 2012|77.599998|
+----------+---------+

#### What is the average Close for each Calendar Month?
#### In other words, across all the years, what is the average Close price for Jan,Feb, Mar, etc... Your result will have a value for each of these months.

In [47]:
from pyspark.sql.functions import month

result03 = df.groupBy(month('Date')).mean('Close').orderBy('month(Date)', ascending = 1)

result03.withColumnRenamed('month(Date)', 'Mês do ano').show()

+----------+-----------------+
Mês do ano| avg(Close)|
+----------+-----------------+
 1|71.44801958415842|
 2| 71.306804443299|
 3|71.77794377570092|
 4|72.97361900952382|
 5|72.30971688679247|
 6| 72.4953774245283|
 7|74.43971943925233|
 8|73.02981855454546|
 9|72.18411785294116|
 10|71.57854545454543|
 11| 72.1110893069307|
 12|72.84792478301885|
+----------+-----------------+

# Great Job!